- **Veri seti**: Titanic veri seti.
- **Amaç**: 
  - `Sex` sütununu veri setinden düşürmek.
  - `Name` sütunundan unvanları (`Title`, örneğin Mr, Mrs, Miss) çıkararak cinsiyet tahmini yapmak.
  - Eksik verileri doldurmak ve yeni özellikler oluşturmak.
  - Bu aşamaya kadar kullanılan kodların fonksiyonlaştırılması.
  - Tüm işlemleri fonksiyonlar halinde düzenlemek.
- **Kullanılan kütüphaneler**: `pandas`, `numpy`.


## Görevler
Aşağıdaki adımları takip ederek özellik mühendisliği yapın. Her bir görev için ayrı bir fonksiyon yazmanız gerekiyor. Fonksiyonlar, modüler ve tekrar kullanılabilir olmalıdır.

### 1. Veri Setini Yükleme
- **Görev**: Titanic veri setini bir pandas DataFrame’e yükleyin.
- **Fonksiyon**: `load_data(file_path)`
  - **Parametre**: `file_path` (dosya yolu, örneğin "train.csv")
  - **Çıktı**: Pandas DataFrame

### 2. Eksik Verileri Doldurma
- **Görev**: `Age` ve `Embarked` sütunlarındaki eksik verileri doldurun.
  - `Age`: Ortalama yaş ile doldurun.
  - `Embarked`: En sık kullanılan liman ile doldurun.
- **Fonksiyon**: `fill_missing_values(df)`
  - **Parametre**: DataFrame
  - **Çıktı**: Eksik verileri doldurulmuş DataFrame
  - **Not**: Orijinal DataFrame’i değiştirmemek için kopyasını döndürün.

### 3. Sex Sütununu Düşürme
- **Görev**: `Sex` sütununu veri setinden kaldırın.
- **Fonksiyon**: `drop_sex_column(df)`
  - **Parametre**: DataFrame
  - **Çıktı**: `Sex` sütunu kaldırılmış DataFrame
  - **Not**: Orijinal DataFrame’i değiştirmemek için kopyasını döndürün.

### 4. Unvan Çıkarımı (Title Extraction)
- **Görev**: `Name` sütunundan unvanları (`Mr`, `Mrs`, `Miss`, `Master` vb.) çıkarın ve yeni bir `Title` sütunu oluşturun.
- **Fonksiyon**: `extract_title(df)`
  - **Parametre**: DataFrame
  - **Çıktı**: `Title` sütunu eklenmiş DataFrame
  - **Not**: Unvanları düzenli ifadeler (regex) kullanarak çıkarın. Örneğin, "Mr." veya "Mrs." gibi ifadeleri arayın.

### 5. Cinsiyet Tahmini
- **Görev**: `Title` sütununa dayalı olarak cinsiyet tahmini yapın ve yeni bir `EstimatedSex` sütunu oluşturun.
  - Örneğin: `Mr`, `Master` → `male`; `Mrs`, `Miss` → `female`; diğer unvanlar için en yaygın cinsiyeti kullanın.
- **Fonksiyon**: `estimate_sex_from_title(df)`
  - **Parametre**: DataFrame
  - **Çıktı**: `EstimatedSex` sütunu eklenmiş DataFrame
  - **Not**: Orijinal DataFrame’i değiştirmemek için kopyasını döndürün.

### 6. Yeni Özellik Oluşturma
- **Görev**: Aşağıdaki yeni özellikleri oluşturun:
  - `FamilySize`: `SibSp` + `Parch` + 1 (kendi dahil)
  - `IsAlone`: `FamilySize` 1 ise 1, değilse 0
  - `AgeGroup`: Yaşı kategorilere ayırın (örneğin, 0-12: Çocuk, 13-19: Genç, 20-59: Yetişkin, 60+: Yaşlı)
- **Fonksiyon**: `create_new_features(df)`
  - **Parametre**: DataFrame
  - **Çıktı**: Yeni özellikler eklenmiş DataFrame
  - **Not**: Orijinal DataFrame’i değiştirmemek için kopyasını döndürün.

### 7. Tüm İşlemleri Birleştirme
- **Görev**: Yukarıdaki tüm işlemleri sırayla uygulayan bir ana fonksiyon yazın.
- **Fonksiyon**: `process_titanic_data(file_path)`
  - **Parametre**: `file_path`
  - **Çıktı**: İşlenmiş DataFrame
  - **Not**: Tüm fonksiyonları sırayla çağırarak veri setini tamamen işleyin.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# 1. Veri Setini Yükleme
# TODO: Dosya varlığını kontrol et ve DataFrame döndür
"""
    Titanic veri setini yükler.
    Parametre: file_path (str) - Veri setinin dosya yolu
    Çıktı: Pandas DataFrame
    """


def load_data(file_path):
    try:
        df=pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return sns.load_dataset("titanic") # eğer dosya bulunamazsa seaborn'dan
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
    
    pass

df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 2. Eksik Verileri Doldurma
# TODO: Age için ortalama, Embarked için en sık değer ile doldur
"""
    Age ve Embarked sütunlarındaki eksik verileri doldurur.
    Parametre: df (DataFrame)
    Çıktı: Eksik verileri doldurulmuş DataFrame
"""
    
    
def fill_missing_values(df): 
    df_copy = df.copy()
    df_copy['Age'] = df_copy['Age'].fillna(df_copy['Age'].mean())
    df_copy['Embarked'] = df_copy['Embarked'].fillna(df_copy['Embarked'].mode()[0])
    return df_copy



In [3]:
# 3. Sex Sütununu Düşürme
# TODO: Sex sütununu kaldır
"""
    Sex sütununu düşürür.
    Parametre: df (DataFrame)
    Çıktı: Sex sütunu kaldırılmış DataFrame
"""  


def drop_sex_column(df):   
    df_copy = df.copy()
    df_copy =  df_copy.drop(columns=['Sex'], errors='ignore')
    return df_copy





In [4]:
# 4. Unvan Çıkarımı
# TODO: Regex ile unvanları çıkar ve Title sütunu ekle
"""
    Name sütunundan unvanları çıkarır ve Title sütunu ekler.
    Parametre: df (DataFrame)
    Çıktı: Title sütunu eklenmiş DataFrame
"""


def extract_title(df):
    df_copy = df.copy()
    if 'Name' in df_copy.columns:
        # Regex ile unvanları çıkar (Mr., Mrs., Miss., Master. vb.)
        df_copy['Title'] = df_copy['Name'].str.extract(r'([A-Za-z]+)\.', expand=False)
        
        # Unvanları temizle ve standartlaştır
        df_copy['Title'] = df_copy['Title'].str.strip()
        
        # Nadir unvanları grupla
        df_copy['Title'] = df_copy['Title'].replace(["Mlle", "Ms", "Mme"], "Miss")
        df_copy['Title'] = df_copy['Title'].replace(["Mrs"], "Mrs")
        df_copy['Title'] = df_copy['Title'].replace(["Dr", "Rev", "Major", "Col", "Don",
                                                     "Sir", "Capt", "Jonkheer", "Lady", "Countess"], "Rare")
       
    return df_copy



In [5]:
# 5. Cinsiyet Tahmini
# TODO: Title'a göre cinsiyet tahmini yap ve EstimatedSex sütunu ekle
"""
    Title sütununa göre cinsiyet tahmini yapar ve EstimatedSex sütunu ekler.
    Parametre: df (DataFrame)
    Çıktı: EstimatedSex sütunu eklenmiş DataFrame
"""
    

def estimate_sex_from_title(df):
    df_copy = df.copy()
    
     # Unvanlara göre cinsiyet tahmini
    title_to_sex = {
        'Mr': 'male',
        'Mrs': 'female',
        'Miss': 'female',
        'Master': 'male',
        'Rare': 'male'  # Nadir unvanlar için varsayılan olarak male
    }
    
    df_copy['EstimatedSex'] = df_copy['Title'].map(title_to_sex)
    
    # Eğer Title sütunu boş ise, varsayılan olarak male ata
    df_copy['EstimatedSex'] = df_copy['EstimatedSex'].fillna('male')
    
    return df_copy



In [6]:
# 6. Yeni Özellik Oluşturma
# TODO: FamilySize, IsAlone ve AgeGroup özelliklerini ekle
"""
    FamilySize, IsAlone ve AgeGroup özelliklerini oluşturur.
    Parametre: df (DataFrame)
    Çıktı: Yeni özellikler eklenmiş DataFrame
"""
    

def create_new_features(df):
    df_copy = df.copy()
    
    # FamilySize özelliği
    df_copy['FamilySize'] = df_copy['SibSp'] + df_copy['Parch'] + 1
    
    # IsAlone özelliği
    df_copy['IsAlone'] = 0
    df_copy.loc[df_copy['FamilySize'] == 1, 'IsAlone'] = 1
    
    # AgeGroup özelliği
    bins = [0, 12, 19, 60, np.inf]
    labels = ['Child', 'Teenager', 'Adult', 'Elderly']
    df_copy['AgeGroup'] = pd.cut(df_copy['Age'], bins=bins, labels=labels)
    
    return df_copy



In [7]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import re

# Pipeline için özel transformer sınıfları
class FillMissingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return fill_missing_values(X)

class DropSexTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return drop_sex_column(X)

class ExtractTitleTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return extract_title(X)

class EstimateSexTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return estimate_sex_from_title(X)

class CreateFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return create_new_features(X)

# Pipeline oluşturma
titanic_pipeline = Pipeline([
    ('fill_missing', FillMissingTransformer()),
    ('extract_title', ExtractTitleTransformer()),
    ('estimate_sex', EstimateSexTransformer()),
    ('drop_sex', DropSexTransformer()),
    ('create_features', CreateFeaturesTransformer())
])

In [8]:
# 7. Tüm İşlemleri Birleştirme
# TODO: Yukarıdaki fonksiyonları sırayla çağır
"""
    Tüm özellik mühendisliği adımlarını sırayla uygular.
    Parametre: file_path (str) - Veri setinin dosya yolu
    Çıktı: İşlenmiş DataFrame
"""


def process_titanic_data(file_path):
    
    df = load_data(file_path)
    if df is None:
      return None
    
# Pipeline kullanarak işlem yapma
    processed_df = titanic_pipeline.fit_transform(df)
    return processed_df

# Test etme
if __name__ == "__main__":
    # Veri setini yükle
    url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
    
    try:
        # Direkt URL'den yükle
        df = pd.read_csv(url)
        print("Veri seti başarıyla yüklendi!")
        print(f"Orijinal veri boyutu: {df.shape}")
        
        # Pipeline ile işle
        processed_data = titanic_pipeline.fit_transform(df)
        
        print("\nVeri başarıyla işlendi!")
        print(f"İşlenmiş veri boyutu: {processed_data.shape}")
        
        print("\nİlk 5 satır:")
        print(processed_data.head(10))
        
        print("\nYeni sütunlar:")
        new_columns = ['Title', 'EstimatedSex', 'FamilySize', 'IsAlone', 'AgeGroup']
        existing_new_columns = [col for col in new_columns if col in processed_data.columns]
        if existing_new_columns:
            print(processed_data[existing_new_columns].head(10))
        
        print("\nEksik veri kontrolü:")
        missing_data = processed_data.isnull().sum()
        print(missing_data[missing_data > 0])
        
        print("\nTitle dağılımı:")
        if 'Title' in processed_data.columns:
            print(processed_data['Title'].value_counts())
        
        print("\nEstimatedSex dağılımı:")
        if 'EstimatedSex' in processed_data.columns:
            print(processed_data['EstimatedSex'].value_counts())
        
        print("\nAgeGroup dağılımı:")
        if 'AgeGroup' in processed_data.columns:
            print(processed_data['AgeGroup'].value_counts())
            
        # Alternatif olarak fonksiyonları tek tek çağırma
        print("\n" + "="*50)
        print("Alternatif: Fonksiyonları tek tek çağırma")
        print("="*50)
        
        df_alt = pd.read_csv(url)
        df_alt = fill_missing_values(df_alt)
        df_alt = extract_title(df_alt)
        df_alt = estimate_sex_from_title(df_alt)
        df_alt = drop_sex_column(df_alt)
        df_alt = create_new_features(df_alt)
        
        print("Alternatif yöntemle işlenmiş veri:")
        print(df_alt[['Title', 'EstimatedSex', 'FamilySize', 'IsAlone', 'AgeGroup']].head())
        
    except Exception as e:
        print(f"Hata oluştu: {e}")
        
        # Seaborn'dan yükle
        print("Seaborn'dan veri yükleniyor...")
        df = sns.load_dataset("titanic")
        processed_data = titanic_pipeline.fit_transform(df)
        print("Seaborn verisi ile işlem tamamlandı!")
        print(processed_data.head())



Veri seti başarıyla yüklendi!
Orijinal veri boyutu: (891, 12)

Veri başarıyla işlendi!
İşlenmiş veri boyutu: (891, 16)

İlk 5 satır:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name        Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris  22.000000      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.000000      1      0   
2                             Heikkinen, Miss. Laina  26.000000      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.000000      1      0   
4                           Allen, Mr. William Henry  35.000000  

- Pipeline süreçleri oluşturun ve ColumnsTransformer kullanın.

- Kendi Fonksiyonlarınızı Pipeline ile kulanmaya çalışın.
- Fonksiyonlarınızı yazarken, her zaman DataFrame’in bir kopyasını döndürerek orijinal veriyi korumaya özen gösterin.
- `EstimatedSex` için unvanların cinsiyetle ilişkisini belirlerken, yaygın unvanlar (`Mr`, `Mrs`, `Miss`, `Master`) için sabit kurallar kullanabilirsiniz. Nadir unvanlar için varsayılan bir cinsiyet atayabilirsiniz (örneğin, `male`).
